In [ ]:
from collections import defaultdict, deque

def readfile(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    sections = content.strip().split('\n\n')
    rules_section = sections[0].strip().split('\n')
    updates_section = sections[1].strip().split('\n')

    return rules_section, updates_section

def rules(rules_lines):
    rules = []
    for line in rules_lines:
        parts = line.strip().split('|')
        A, B = parts
        rules.append((int(A), int(B)))
    return rules

def updates(updates_lines):
    updates = []
    for line in updates_lines:
        update = [int(page) for page in line.split(',')]
        updates.append(update)
    return updates

def is_update_compliant(update, rules):
    # Create a dictionary to map page number to its index in the update
    page_position = {page: idx for idx, page in enumerate(update)}

    for A, B in rules:
        if A in page_position and B in page_position:
            if page_position[A] >= page_position[B]:
                return False
    return True

def filter_compliant_and_non_compliant_updates(updates, rules):
    compliant = []
    non_compliant = []
    for update in updates:
        if is_update_compliant(update, rules):
            compliant.append(update)
        else:
            non_compliant.append(update)
    return compliant, non_compliant

def build_graph(rules, update_pages):
    """
    Builds a graph based on the rules for the pages present in the update.
    """
    graph = defaultdict(list)
    in_degree = defaultdict(int)
    
    # Initialize in_degree for all pages in the update
    for page in update_pages:
        in_degree[page] = 0

    for A, B in rules:
        if A in update_pages and B in update_pages:
            graph[A].append(B)
            in_degree[B] += 1

    return graph, in_degree

def topological_sort(pages, graph, in_degree):
    """
    Performs topological sorting on the given pages based on the graph.
    Returns the sorted list if possible, else None.
    """
    queue = deque()
    for page in pages:
        if in_degree[page] == 0:
            queue.append(page)

    sorted_order = []
    while queue:
        current = queue.popleft()
        sorted_order.append(current)
        for neighbor in graph[current]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)

    if len(sorted_order) != len(pages):
        # Cycle detected or not all pages processed
        return None
    return sorted_order

def correct_update(update, rules):
    """
    Reorders the update to comply with the ordering rules using topological sort.
    If multiple valid orderings are possible, it preserves the original relative order as much as possible.
    """
    graph, in_degree = build_graph(rules, update)
    
    # Attempt topological sort
    sorted_update = topological_sort(update, graph, in_degree)
    
    if sorted_update is not None:
        return sorted_update
    else:
        # If cycle detected or not sortable, return original update
        return update

def find_middle_number(update):
    if len(update) == 0:
        return None
    middle_index = len(update) // 2
    if len(update) % 2 == 0:
        return update[middle_index - 1]
    else:
        return update[middle_index]

def calculate_middle_sum(updates):
    middle_numbers = [find_middle_number(update) for update in updates if find_middle_number(update) is not None]
    return sum(middle_numbers)

input_file = r'C:\Users\91630\Downloads\AOC\AOCday5\AOC5_Input.txt'

# Step 1: Read and parse the input file
rules_lines, updates_lines = readfile(input_file)

# Step 2: Parse rules and updates
rules = rules(rules_lines)
updates = updates(updates_lines)

# Step 3: Filter compliant and non-compliant updates
compliant_updates, non_compliant_updates = filter_compliant_and_non_compliant_updates(updates, rules)

# Step 4: Correct non-compliant updates
corrected_updates = [correct_update(update, rules) for update in non_compliant_updates]

# Step 5: Calculate middle sum for compliant updates
compliant_middle_sum = calculate_middle_sum(compliant_updates)
print(f"The sum of all middle numbers from the compliant updates is: {compliant_middle_sum}")

# Step 6: Calculate middle sum for corrected non-compliant updates
corrected_middle_sum = calculate_middle_sum(corrected_updates)
print(f"The sum of all middle numbers from the corrected non-compliant updates is: {corrected_middle_sum}")

The sum of all middle numbers from the compliant updates is: 7307
The sum of all middle numbers from the corrected non-compliant updates is: 4713
